In [ ]:
# =========================
# Step 0: 基本設定與資料讀取
# =========================

print("="*80)
print("Step 0: 基本設定與資料讀取")
print("="*80)

# 0-1. 匯入所有需要的套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from pathlib import Path
import os

# 機器學習相關
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix,
    f1_score, precision_score, recall_score,
    roc_auc_score, precision_recall_curve,
    ConfusionMatrixDisplay
)

# 處理不平衡資料
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# 模型
import xgboost as xgb
import lightgbm as lgb

# 超參數調整
import optuna

# 模型解釋
import shap
from lime import lime_tabular

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# 0-2. 設定隨機種子，確保結果可重現
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 0-2.1 設定 Optuna 調參次數（可自行調整）⭐
OPTUNA_TRIALS = 20  # 🔧 調整這個數字來控制 Optuna 的搜索次數
print(f"\n⚙️ Optuna 調參次數設定: {OPTUNA_TRIALS} trials")
print("   💡 提示：")
print("      - 10 trials: 快速測試（約 5-10 分鐘）")
print("      - 20 trials: 平衡選擇（約 10-20 分鐘）⭐ 推薦")
print("      - 50 trials: 深度搜索（約 25-40 分鐘）")
print("      - 100 trials: 極致優化（約 50-80 分鐘）")

# 0-3. 設定輸出路徑
OUTPUT_DIR = '/Users/'
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("\n✓ 所有套件匯入成功")
print(f"✓ 隨機種子設定為: {RANDOM_STATE}")
print(f"✓ 輸出路徑: {OUTPUT_DIR}")

# 0-3. 讀取資料
DATA_PATH = '/Users/creditcard.csv'

print(f"\n正在讀取資料: {DATA_PATH}")
df = pd.read_csv(DATA_PATH)
print("✓ 資料讀取完成")

# 0-4. 顯示基本資訊
print("\n" + "="*80)
print("資料基本資訊")
print("="*80)

# 資料列數與欄位數
print(f"\n資料形狀: {df.shape}")
print(f"  - 總交易筆數: {df.shape[0]:,}")
print(f"  - 總欄位數: {df.shape[1]}")

# 前 5 筆資料
print("\n前 5 筆資料:")
print(df.head())

# 各欄位的缺失值情況
print("\n缺失值檢查:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✓ 沒有缺失值")
else:
    print(missing[missing > 0])

# Class 的類別比例
print("\nClass 類別分布:")
class_counts = df['Class'].value_counts()
class_ratio = df['Class'].value_counts(normalize=True)

print(f"  正常交易 (Class=0): {class_counts[0]:,} ({class_ratio[0]*100:.3f}%)")
print(f"  詐欺交易 (Class=1): {class_counts[1]:,} ({class_ratio[1]*100:.3f}%)")
print(f"  不平衡比例: {class_counts[0]/class_counts[1]:.1f}:1")

# =========================
# Step 1: 初步 EDA
# =========================

print("\n" + "="*80)
print("Step 1: 初步探索性資料分析 (EDA)")
print("="*80)

# 1-1. Time 的視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time 直方圖
axes[0].hist(df['Time'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Time Distribution (Histogram)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (seconds)')
axes[0].set_ylabel('Frequency')
axes[0].grid(alpha=0.3)

# Time boxplot
axes[1].boxplot(df['Time'], vert=True)
axes[1].set_title('Time Distribution (Boxplot)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Time (seconds)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '01_time_distribution.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"\n✓ Time 視覺化完成 (已存檔: {OUTPUT_DIR}/01_time_distribution.png)")

# 1-2. Amount 的視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Amount 直方圖
axes[0].hist(df['Amount'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Amount Distribution (Histogram)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Amount')
axes[0].set_ylabel('Frequency')
axes[0].grid(alpha=0.3)

# Amount boxplot
axes[1].boxplot(df['Amount'], vert=True)
axes[1].set_title('Amount Distribution (Boxplot)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Amount')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '02_amount_distribution.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Amount 視覺化完成 (已存檔: {OUTPUT_DIR}/02_amount_distribution.png)")

# 1-3. 輸出觀察結論
print("\n" + "-"*80)
print("EDA 觀察結論:")
print("-"*80)

print("\n關於 Time:")
print(f"  - 範圍: {df['Time'].min():.0f} ~ {df['Time'].max():.0f} 秒")
print(f"  - 平均: {df['Time'].mean():.0f} 秒")
print(f"  - 中位數: {df['Time'].median():.0f} 秒")
print("  - 觀察: Time 顯示兩個明顯的交易高峰期，可能代表不同天或時段")

print("\nAmount:")
print(f"  - 範圍: ${df['Amount'].min():.2f} ~ ${df['Amount'].max():.2f}")
print(f"  - 平均: ${df['Amount'].mean():.2f}")
print(f"  - 中位數: ${df['Amount'].median():.2f}")
print(f"  - 95% 分位數: ${df['Amount'].quantile(0.95):.2f}")
print(f"  - 99% 分位數: ${df['Amount'].quantile(0.99):.2f}")
print("  - 觀察: Amount 非常偏態，有長尾分布。大部分交易金額很小，少數交易金額很大")
print("  - 建議: 應該對 Amount 做對數轉換 (log transformation)")

# =========================
# Step 2: 特徵工程 (不刪欄位，只新增)
# =========================

print("\n" + "="*80)
print("Step 2: 特徵工程 (Feature Engineering)")
print("="*80)
print("\n重要: 我們只新增特徵，不刪除任何原始欄位")

# 複製一份資料以避免修改原始資料
df_fe = df.copy()

# 2-1. Time 相關特徵
print("\n處理 Time 特徵...")

# time_diff: 當前交易與前一筆交易的時間差
df_fe['time_diff'] = df_fe['Time'].diff().fillna(0)
print("  ✓ 新增 time_diff (與前一筆交易的時間差)")

# time_sin, time_cos: 將 Time 當作週期性數值
df_fe['time_norm'] = df_fe['Time'] / df_fe['Time'].max()
df_fe['time_sin'] = np.sin(2 * np.pi * df_fe['time_norm'])
df_fe['time_cos'] = np.cos(2 * np.pi * df_fe['time_norm'])
print("  ✓ 新增 time_sin, time_cos (週期性特徵)")

# 2-2. Amount 相關特徵
print("\n處理 Amount 特徵...")

# log_amount: log1p 轉換（處理偏態分布）
df_fe['log_amount'] = np.log1p(df_fe['Amount'])
print("  ✓ 新增 log_amount (對數轉換)")

# 💡 V1~V28 是 PCA 後的特徵，已經標準化，不需要額外 scaling
print("\n💡 V1~V28 是 PCA 後的特徵，已經標準化，不需要額外 scaling")

# =========================
# Step 2.5: 策略性特徵工程（基於 SHAP Top 特徵）⭐
# =========================

print("\n" + "="*80)
print("Step 2.5: 策略性特徵工程（基於 SHAP 分析）")
print("="*80)

print("\n📊 根據 SHAP 分析，最重要的特徵為：")
print("  1. V14 (重要性 ~5.0) ⭐⭐⭐")
print("  2. V4  (重要性 ~2.0) ⭐⭐")
print("  3. V12 (重要性 ~1.2) ⭐")
print("  4. V10 (重要性 ~1.1) ⭐")
print("  5-10. V11, V3, V17, log_amount, V9, V1")

print("\n💡 策略：只針對 TOP 特徵做精準加工，而非亂加 50 個欄位")

# (1) log_amount 分箱 - 樹模型喜歡離散特徵
print("\n 1️⃣ log_amount 分箱（離散化）")
df_fe['amount_bin'] = pd.qcut(df_fe['log_amount'], q=4, labels=False, duplicates='drop')
print("  ✓ 新增 amount_bin (分成 4 個等級)")
print("    0=很便宜, 1=中等, 2=偏高, 3=非常高")

# (2) 交互特徵 - 捕捉重要特徵與金額的組合效應
print("\n 2️⃣ 交互特徵（重要特徵 × log_amount）")

# V14 × log_amount：V14 是絕對王者，與金額的交互可能揭示詐欺模式
df_fe['V14_x_logA'] = df_fe['V14'] * df_fe['log_amount']
print("  ✓ 新增 V14_x_logA (V14 × log_amount)")
print("    → V14 奇怪 pattern × 大金額 = 更可疑")

# V4 × log_amount：V4 排名第二
df_fe['V4_x_logA'] = df_fe['V4'] * df_fe['log_amount']
print("  ✓ 新增 V4_x_logA (V4 × log_amount)")

# V12 × log_amount：V12 排名第三
df_fe['V12_x_logA'] = df_fe['V12'] * df_fe['log_amount']
print("  ✓ 新增 V12_x_logA (V12 × log_amount)")

# (3) TOP 兩個特徵的交互
print("\n 3️⃣ 頂級特徵交互（V14 × V4）")
df_fe['V14_x_V4'] = df_fe['V14'] * df_fe['V4']
print("  ✓ 新增 V14_x_V4 (兩個最重要特徵的組合)")
print("    → 捕捉兩個主要 pattern 的聯合效應")

# (4) 極端值標記 - 捕捉異常模式
print("\n 4️⃣ 極端值標記（異常偵測）")

# V14 的極端值（根據 SHAP 圖，V14 的範圍很大，從 -8 到 +2）
v14_q01 = df_fe['V14'].quantile(0.01)
v14_q99 = df_fe['V14'].quantile(0.99)
df_fe['V14_extreme'] = ((df_fe['V14'] < v14_q01) | (df_fe['V14'] > v14_q99)).astype(int)
print(f"  ✓ 新增 V14_extreme (V14 < {v14_q01:.2f} 或 > {v14_q99:.2f})")

# V4 的極端值
v4_q01 = df_fe['V4'].quantile(0.01)
v4_q99 = df_fe['V4'].quantile(0.99)
df_fe['V4_extreme'] = ((df_fe['V4'] < v4_q01) | (df_fe['V4'] > v4_q99)).astype(int)
print(f"  ✓ 新增 V4_extreme (V4 < {v4_q01:.2f} 或 > {v4_q99:.2f})")

# (5) 高金額交易標記
print("\n 5️⃣ 高金額交易標記")
amount_q95 = df_fe['log_amount'].quantile(0.95)
df_fe['high_amount'] = (df_fe['log_amount'] > amount_q95).astype(int)
print(f"  ✓ 新增 high_amount (log_amount > {amount_q95:.2f}，即 top 5%)")

print("\n" + "-"*80)
print("策略性特徵工程總結:")
print("-"*80)
print("新增特徵數量: 8 個（精簡但有效）")
print("\n類型分布:")
print("  - 離散特徵: 1 個 (amount_bin)")
print("  - 交互特徵: 4 個 (V14_x_logA, V4_x_logA, V12_x_logA, V14_x_V4)")
print("  - 極端值標記: 3 個 (V14_extreme, V4_extreme, high_amount)")
print("\n✅ 優點：")
print("  1. 只針對 SHAP Top 特徵，避免過度複雜")
print("  2. 包含離散、交互、極端值三種類型")
print("  3. 樹模型友善（離散值、標記）")
print("  4. 捕捉極端 / 奇怪的組合模式")
print("="*80)

# 檢查新增的特徵
print("\n" + "-"*80)
print("特徵工程總結:")
print("-"*80)
print(f"原始欄位數: {df.shape[1]}")
print(f"新增後欄位數: {df_fe.shape[1]}")
print(f"新增特徵數: {df_fe.shape[1] - df.shape[1]}")

print("\n新增的特徵:")
new_features = [col for col in df_fe.columns if col not in df.columns]

# 分類顯示
basic_features = [f for f in new_features if f in ['time_diff', 'time_norm', 'time_sin', 'time_cos', 'log_amount']]
strategic_features = [f for f in new_features if f not in basic_features]

print("\n基礎特徵（5 個）:")
for i, feat in enumerate(basic_features, 1):
    print(f"  {i}. {feat}")

print("\n策略性特徵（8 個，基於 SHAP）⭐:")
for i, feat in enumerate(strategic_features, 1):
    print(f"  {i}. {feat}")

print("\n💡 策略性特徵說明:")
print("  - amount_bin: log_amount 的 4 分位離散化")
print("  - V14_x_logA, V4_x_logA, V12_x_logA: 重要特徵與金額的交互")
print("  - V14_x_V4: 兩個最重要特徵的組合")
print("  - V14_extreme, V4_extreme: 極端值標記")
print("  - high_amount: 高金額交易標記（top 5%）")

# =========================
# Step 3: 切分訓練集、驗證集與測試集 (嚴謹版本)
# =========================

print("\n" + "="*80)
print("Step 3: 切分訓練集、驗證集與測試集")
print("="*80)

# 3-1. 定義特徵 X 和目標變數 y
X = df_fe.drop('Class', axis=1)
y = df_fe['Class']

print(f"\n特徵矩陣 X: {X.shape}")
print(f"目標變數 y: {y.shape}")

# 3-2. 先切分出測試集 (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

# 3-3. 再從剩餘的 80% 中切分出驗證集 (實際佔總資料的 20%)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp,
    test_size=0.25,  # 0.25 * 0.8 = 0.2 of total
    stratify=y_temp,
    random_state=RANDOM_STATE
)

print(f"\n資料切分完成:")
print(f"  訓練集: {X_train.shape[0]:,} 筆 ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  驗證集: {X_valid.shape[0]:,} 筆 ({X_valid.shape[0]/len(X)*100:.1f}%)")
print(f"  測試集: {X_test.shape[0]:,} 筆 ({X_test.shape[0]/len(X)*100:.1f}%)")

# 檢查類別比例
print(f"\n訓練集類別分布:")
train_class_counts = y_train.value_counts()
print(f"  正常交易: {train_class_counts[0]:,} ({train_class_counts[0]/len(y_train)*100:.3f}%)")
print(f"  詐欺交易: {train_class_counts[1]:,} ({train_class_counts[1]/len(y_train)*100:.3f}%)")

print(f"\n驗證集類別分布:")
valid_class_counts = y_valid.value_counts()
print(f"  正常交易: {valid_class_counts[0]:,} ({valid_class_counts[0]/len(y_valid)*100:.3f}%)")
print(f"  詐欺交易: {valid_class_counts[1]:,} ({valid_class_counts[1]/len(y_valid)*100:.3f}%)")

print(f"\n測試集類別分布:")
test_class_counts = y_test.value_counts()
print(f"  正常交易: {test_class_counts[0]:,} ({test_class_counts[0]/len(y_test)*100:.3f}%)")
print(f"  詐欺交易: {test_class_counts[1]:,} ({test_class_counts[1]/len(y_test)*100:.3f}%)")

print("\n💡 V1~V28 是 PCA 後的特徵，已經標準化過，不需要額外 scaling")

# =========================
# Step 4: 處理類別不平衡
# =========================

print("\n" + "="*80)
print("Step 4: 處理類別不平衡")
print("="*80)
print("\n⚠️ 重要: 只對訓練集做 resampling，驗證集和測試集保持原本不平衡")

# 4-1. 使用 SMOTE
print("\n使用 SMOTE 處理訓練集:")
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"  原始訓練集: {len(y_train):,} 筆")
print(f"  SMOTE 後: {len(y_train_smote):,} 筆")
print(f"  詐欺交易數量: {(y_train==1).sum()} → {(y_train_smote==1).sum()}")
print(f"  正常交易數量: {(y_train==0).sum()} → {(y_train_smote==0).sum()}")
print(f"  新的比例: {(y_train_smote==0).sum()}:{(y_train_smote==1).sum()} (已平衡)")

# =========================
# Step 5: 建立 Baseline 模型 + Optuna 調參
# =========================

print("\n" + "="*80)
print("Step 5: 模型訓練")
print("="*80)

# 5-1. Logistic Regression Baseline ⭐
print("\n" + "-"*80)
print("Step 5.1: Logistic Regression Baseline")
print("-"*80)

print("\n訓練 Logistic Regression 作為 baseline...")

# 使用 SMOTE 訓練集
lr_baseline = LogisticRegression(
    max_iter=1000,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

lr_baseline.fit(X_train_smote, y_train_smote)
print("✓ Logistic Regression 訓練完成")

# 在驗證集上評估
y_valid_pred_lr = lr_baseline.predict(X_valid)
y_valid_proba_lr = lr_baseline.predict_proba(X_valid)[:, 1]

lr_precision = precision_score(y_valid, y_valid_pred_lr, pos_label=1)
lr_recall = recall_score(y_valid, y_valid_pred_lr, pos_label=1)
lr_f1 = f1_score(y_valid, y_valid_pred_lr, pos_label=1)
lr_roc_auc = roc_auc_score(y_valid, y_valid_proba_lr)

print("\n驗證集性能 (Logistic Regression Baseline):")
print(f"  Precision: {lr_precision:.4f}")
print(f"  Recall: {lr_recall:.4f}")
print(f"  F1-Score: {lr_f1:.4f}")
print(f"  ROC-AUC: {lr_roc_auc:.4f}")

# 5-2. Optuna 調參 - LightGBM
print("\n" + "-"*80)
print("Step 5.2: LightGBM 超參數調整")
print("-"*80)

def objective_lgb(trial):
    """LightGBM 的 Optuna objective 函數"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbose': -1
    }
    
    model = lgb.LGBMClassifier(**params)
    
    # 使用 StratifiedKFold 交叉驗證
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    f1_scores = []
    
    for train_idx, val_idx in cv.split(X_train_smote, y_train_smote):
        X_tr = X_train_smote.iloc[train_idx]
        X_val = X_train_smote.iloc[val_idx]
        y_tr = y_train_smote.iloc[train_idx]
        y_val = y_train_smote.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, pos_label=1)
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

print("\n開始 Optuna 優化...")
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n✓ LightGBM 調參完成 ({OPTUNA_TRIALS} trials)")
print(f"  最佳 F1-Score (CV): {study_lgb.best_value:.4f}")
print(f"  最佳參數:")
for key, value in study_lgb.best_params.items():
    print(f"    {key}: {value}")

# 訓練最佳 LightGBM 模型並在驗證集上評估
best_lgb = lgb.LGBMClassifier(**study_lgb.best_params, random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)
best_lgb.fit(X_train_smote, y_train_smote)
y_valid_pred_lgb = best_lgb.predict(X_valid)
y_valid_proba_lgb = best_lgb.predict_proba(X_valid)[:, 1]

lgb_precision = precision_score(y_valid, y_valid_pred_lgb, pos_label=1)
lgb_recall = recall_score(y_valid, y_valid_pred_lgb, pos_label=1)
lgb_f1 = f1_score(y_valid, y_valid_pred_lgb, pos_label=1)
lgb_roc_auc = roc_auc_score(y_valid, y_valid_proba_lgb)

print(f"\n驗證集性能 (LightGBM):")
print(f"  Precision: {lgb_precision:.4f}")
print(f"  Recall: {lgb_recall:.4f}")
print(f"  F1-Score: {lgb_f1:.4f}")
print(f"  ROC-AUC: {lgb_roc_auc:.4f}")

# 5-3. Optuna 調參 - XGBoost
print("\n" + "-"*80)
print("Step 5.3: XGBoost 超參數調整")
print("-"*80)

def objective_xgb(trial):
    """XGBoost 的 Optuna objective 函數"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    
    model = xgb.XGBClassifier(**params)
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    f1_scores = []
    
    for train_idx, val_idx in cv.split(X_train_smote, y_train_smote):
        X_tr = X_train_smote.iloc[train_idx]
        X_val = X_train_smote.iloc[val_idx]
        y_tr = y_train_smote.iloc[train_idx]
        y_val = y_train_smote.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, pos_label=1)
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

print("\n開始 Optuna 優化...")

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n✓ XGBoost 調參完成 ({OPTUNA_TRIALS} trials)")
print(f"  最佳 F1-Score (CV): {study_xgb.best_value:.4f}")
print(f"  最佳參數:")
for key, value in study_xgb.best_params.items():
    print(f"    {key}: {value}")

# 訓練最佳 XGBoost 模型並在驗證集上評估
best_xgb = xgb.XGBClassifier(**study_xgb.best_params, random_state=RANDOM_STATE, n_jobs=-1,
                              use_label_encoder=False, eval_metric='logloss')
best_xgb.fit(X_train_smote, y_train_smote)
y_valid_pred_xgb = best_xgb.predict(X_valid)
y_valid_proba_xgb = best_xgb.predict_proba(X_valid)[:, 1]

xgb_precision = precision_score(y_valid, y_valid_pred_xgb, pos_label=1)
xgb_recall = recall_score(y_valid, y_valid_pred_xgb, pos_label=1)
xgb_f1 = f1_score(y_valid, y_valid_pred_xgb, pos_label=1)
xgb_roc_auc = roc_auc_score(y_valid, y_valid_proba_xgb)

print(f"\n驗證集性能 (XGBoost):")
print(f"  Precision: {xgb_precision:.4f}")
print(f"  Recall: {xgb_recall:.4f}")
print(f"  F1-Score: {xgb_f1:.4f}")
print(f"  ROC-AUC: {xgb_roc_auc:.4f}")

# =========================
# Step 5.5: 不用 SMOTE 的版本（使用 class_weight）⭐⭐⭐
# =========================

print("\n" + "="*80)
print("Step 5.5: 不用 SMOTE 的版本（使用 class_weight）⭐")
print("="*80)

print("\n💡 為什麼嘗試不用 SMOTE？")
print("  - SMOTE 會「捏出假的詐欺樣本」，可能學到人造規律")
print("  - 直接用原始不平衡分布 + 權重，模型在真實比例下學習")
print("  - 實務中，class_weight 版本常常表現更穩定")

# 計算 scale_pos_weight
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos_weight = neg_count / pos_count

print(f"\n資料統計:")
print(f"  正常交易 (Class=0): {neg_count:,}")
print(f"  詐欺交易 (Class=1): {pos_count:,}")
print(f"  scale_pos_weight: {scale_pos_weight:.2f}")

# 5.5.1 LightGBM with class_weight
print("\n" + "-"*80)
print("Step 5.5.1: LightGBM (class_weight='balanced')")
print("-"*80)

def objective_lgb_weighted(trial):
    """LightGBM with class_weight"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'class_weight': 'balanced',  # ⭐ 關鍵差異
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbose': -1
    }
    
    model = lgb.LGBMClassifier(**params)
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    f1_scores = []
    
    for train_idx, val_idx in cv.split(X_train, y_train):  # ⭐ 用原始不平衡資料
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, pos_label=1)
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

print("\n開始 Optuna 優化（不用 SMOTE，使用 class_weight）...")
study_lgb_weighted = optuna.create_study(direction='maximize')
study_lgb_weighted.optimize(objective_lgb_weighted, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n✓ LightGBM (class_weight) 調參完成 ({OPTUNA_TRIALS} trials)")
print(f"  最佳 F1-Score (CV): {study_lgb_weighted.best_value:.4f}")

# 訓練最佳模型
best_lgb_weighted = lgb.LGBMClassifier(**study_lgb_weighted.best_params, 
                                        class_weight='balanced',
                                        random_state=RANDOM_STATE, 
                                        n_jobs=-1, 
                                        verbose=-1)
best_lgb_weighted.fit(X_train, y_train)  # ⭐ 用原始不平衡資料
y_valid_pred_lgb_weighted = best_lgb_weighted.predict(X_valid)
y_valid_proba_lgb_weighted = best_lgb_weighted.predict_proba(X_valid)[:, 1]

lgb_weighted_precision = precision_score(y_valid, y_valid_pred_lgb_weighted, pos_label=1)
lgb_weighted_recall = recall_score(y_valid, y_valid_pred_lgb_weighted, pos_label=1)
lgb_weighted_f1 = f1_score(y_valid, y_valid_pred_lgb_weighted, pos_label=1)
lgb_weighted_roc_auc = roc_auc_score(y_valid, y_valid_proba_lgb_weighted)

print(f"\n驗證集性能 (LightGBM with class_weight):")
print(f"  Precision: {lgb_weighted_precision:.4f}")
print(f"  Recall: {lgb_weighted_recall:.4f}")
print(f"  F1-Score: {lgb_weighted_f1:.4f}")
print(f"  ROC-AUC: {lgb_weighted_roc_auc:.4f}")

# 5.5.2 XGBoost with scale_pos_weight
print("\n" + "-"*80)
print("Step 5.5.2: XGBoost (scale_pos_weight)")
print("-"*80)

def objective_xgb_weighted(trial):
    """XGBoost with scale_pos_weight"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'scale_pos_weight': scale_pos_weight,  # ⭐ 關鍵差異
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    
    model = xgb.XGBClassifier(**params)
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    f1_scores = []
    
    for train_idx, val_idx in cv.split(X_train, y_train):  # ⭐ 用原始不平衡資料
        X_tr = X_train.iloc[train_idx]
        X_val = X_train.iloc[val_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]
        
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, pos_label=1)
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

print("\n開始 Optuna 優化（不用 SMOTE，使用 scale_pos_weight）...")
study_xgb_weighted = optuna.create_study(direction='maximize')
study_xgb_weighted.optimize(objective_xgb_weighted, n_trials=OPTUNA_TRIALS, show_progress_bar=True)

print(f"\n✓ XGBoost (scale_pos_weight) 調參完成 ({OPTUNA_TRIALS} trials)")
print(f"  最佳 F1-Score (CV): {study_xgb_weighted.best_value:.4f}")

# 訓練最佳模型
best_xgb_weighted = xgb.XGBClassifier(**study_xgb_weighted.best_params,
                                       scale_pos_weight=scale_pos_weight,
                                       random_state=RANDOM_STATE,
                                       n_jobs=-1,
                                       use_label_encoder=False,
                                       eval_metric='logloss')
best_xgb_weighted.fit(X_train, y_train)  # ⭐ 用原始不平衡資料
y_valid_pred_xgb_weighted = best_xgb_weighted.predict(X_valid)
y_valid_proba_xgb_weighted = best_xgb_weighted.predict_proba(X_valid)[:, 1]

xgb_weighted_precision = precision_score(y_valid, y_valid_pred_xgb_weighted, pos_label=1)
xgb_weighted_recall = recall_score(y_valid, y_valid_pred_xgb_weighted, pos_label=1)
xgb_weighted_f1 = f1_score(y_valid, y_valid_pred_xgb_weighted, pos_label=1)
xgb_weighted_roc_auc = roc_auc_score(y_valid, y_valid_proba_xgb_weighted)

print(f"\n驗證集性能 (XGBoost with scale_pos_weight):")
print(f"  Precision: {xgb_weighted_precision:.4f}")
print(f"  Recall: {xgb_weighted_recall:.4f}")
print(f"  F1-Score: {xgb_weighted_f1:.4f}")
print(f"  ROC-AUC: {xgb_weighted_roc_auc:.4f}")

# 5-6. 比較所有模型（包括 SMOTE 和 class_weight 版本）
print("\n" + "="*80)
print("模型比較 (驗證集):")
print("="*80)

print("\n📊 A/B 比較：SMOTE vs class_weight")
print("-" * 80)

# 創建完整的比較表格
results_comparison = pd.DataFrame({
    'Model': [
        'Logistic Regression', 
        'LightGBM (SMOTE)', 
        'XGBoost (SMOTE)',
        'LightGBM (class_weight) ⭐',
        'XGBoost (scale_pos_weight) ⭐'
    ],
    'Method': [
        'Baseline',
        'SMOTE',
        'SMOTE',
        'class_weight',
        'scale_pos_weight'
    ],
    'Precision': [
        lr_precision, 
        lgb_precision, 
        xgb_precision,
        lgb_weighted_precision,
        xgb_weighted_precision
    ],
    'Recall': [
        lr_recall, 
        lgb_recall, 
        xgb_recall,
        lgb_weighted_recall,
        xgb_weighted_recall
    ],
    'F1-Score': [
        lr_f1, 
        lgb_f1, 
        xgb_f1,
        lgb_weighted_f1,
        xgb_weighted_f1
    ],
    'ROC-AUC': [
        lr_roc_auc, 
        lgb_roc_auc, 
        xgb_roc_auc,
        lgb_weighted_roc_auc,
        xgb_weighted_roc_auc
    ]
})

print("\n" + results_comparison.to_string(index=False))

# A/B 比較分析
print("\n" + "="*80)
print("A/B 比較分析：")
print("="*80)

print(f"\nLightGBM:")
print(f"  SMOTE 版本:        F1 = {lgb_f1:.4f}")
print(f"  class_weight 版本: F1 = {lgb_weighted_f1:.4f}")
if lgb_weighted_f1 > lgb_f1:
    print(f"  → class_weight 版本較好！ (+{(lgb_weighted_f1 - lgb_f1)*100:.2f}%)")
else:
    print(f"  → SMOTE 版本較好！ (+{(lgb_f1 - lgb_weighted_f1)*100:.2f}%)")

print(f"\nXGBoost:")
print(f"  SMOTE 版本:             F1 = {xgb_f1:.4f}")
print(f"  scale_pos_weight 版本: F1 = {xgb_weighted_f1:.4f}")
if xgb_weighted_f1 > xgb_f1:
    print(f"  → scale_pos_weight 版本較好！ (+{(xgb_weighted_f1 - xgb_f1)*100:.2f}%)")
else:
    print(f"  → SMOTE 版本較好！ (+{(xgb_f1 - xgb_weighted_f1)*100:.2f}%)")

# 視覺化模型比較（更新為 5 個模型）
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

model_names_short = ['LR', 'LGB\n(SMOTE)', 'XGB\n(SMOTE)', 'LGB\n(weight)', 'XGB\n(weight)']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6']

# Precision 比較
axes[0, 0].bar(model_names_short, results_comparison['Precision'], color=colors)
axes[0, 0].set_title('Precision Comparison (All Models)', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Precision', fontsize=12)
axes[0, 0].set_ylim([0.7, 1.0])
axes[0, 0].grid(alpha=0.3, axis='y')
for i, v in enumerate(results_comparison['Precision']):
    axes[0, 0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)

# Recall 比較
axes[0, 1].bar(model_names_short, results_comparison['Recall'], color=colors)
axes[0, 1].set_title('Recall Comparison (All Models)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Recall', fontsize=12)
axes[0, 1].set_ylim([0.7, 1.0])
axes[0, 1].grid(alpha=0.3, axis='y')
for i, v in enumerate(results_comparison['Recall']):
    axes[0, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)

# F1-Score 比較
axes[1, 0].bar(model_names_short, results_comparison['F1-Score'], color=colors)
axes[1, 0].set_title('F1-Score Comparison (All Models)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('F1-Score', fontsize=12)
axes[1, 0].set_ylim([0.7, 1.0])
axes[1, 0].grid(alpha=0.3, axis='y')
for i, v in enumerate(results_comparison['F1-Score']):
    axes[1, 0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)

# ROC-AUC 比較
axes[1, 1].bar(model_names_short, results_comparison['ROC-AUC'], color=colors)
axes[1, 1].set_title('ROC-AUC Comparison (All Models)', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('ROC-AUC', fontsize=12)
axes[1, 1].set_ylim([0.7, 1.0])
axes[1, 1].grid(alpha=0.3, axis='y')
for i, v in enumerate(results_comparison['ROC-AUC']):
    axes[1, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '03_model_comparison_all.png'), dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ 完整模型比較圖已保存: {OUTPUT_DIR}/03_model_comparison_all.png")

# 選擇最佳模型（從所有 5 個模型中）
all_f1_scores = [lr_f1, lgb_f1, xgb_f1, lgb_weighted_f1, xgb_weighted_f1]
all_models = [
    lr_baseline, 
    best_lgb, 
    best_xgb, 
    best_lgb_weighted, 
    best_xgb_weighted
]
all_model_names = [
    'Logistic Regression',
    'LightGBM (SMOTE)',
    'XGBoost (SMOTE)',
    'LightGBM (class_weight)',
    'XGBoost (scale_pos_weight)'
]

best_model_idx = np.argmax(all_f1_scores)
best_model_name = all_model_names[best_model_idx]
final_model = all_models[best_model_idx]

print(f"\n🏆 選擇 {best_model_name} 作為最佳模型")
print(f"   驗證集 F1-Score: {all_f1_scores[best_model_idx]:.4f}")

# =========================
# Step 5.7: Ensemble 模型（LightGBM + XGBoost）⭐⭐⭐
# =========================

print("\n" + "="*80)
print("Step 5.7: Ensemble 模型（LightGBM + XGBoost）⭐")
print("="*80)

print("\n💡 為什麼使用 Ensemble？")
print("  - 結合多個模型的優勢")
print("  - 降低單一模型的偏差")
print("  - 通常能獲得更穩定的預測")

# 5.7.1 準備 Ensemble 的基礎模型
print("\n" + "-"*80)
print("Step 5.7.1: 準備 Ensemble 基礎模型")
print("-"*80)

# 選擇表現最好的 LightGBM 和 XGBoost
if lgb_weighted_f1 > lgb_f1:
    ensemble_lgb = best_lgb_weighted
    lgb_ensemble_name = "LightGBM (class_weight)"
    lgb_ensemble_f1 = lgb_weighted_f1
    y_valid_proba_lgb_ensemble = y_valid_proba_lgb_weighted
else:
    ensemble_lgb = best_lgb
    lgb_ensemble_name = "LightGBM (SMOTE)"
    lgb_ensemble_f1 = lgb_f1
    y_valid_proba_lgb_ensemble = y_valid_proba_lgb

if xgb_weighted_f1 > xgb_f1:
    ensemble_xgb = best_xgb_weighted
    xgb_ensemble_name = "XGBoost (scale_pos_weight)"
    xgb_ensemble_f1 = xgb_weighted_f1
    y_valid_proba_xgb_ensemble = y_valid_proba_xgb_weighted
else:
    ensemble_xgb = best_xgb
    xgb_ensemble_name = "XGBoost (SMOTE)"
    xgb_ensemble_f1 = xgb_f1
    y_valid_proba_xgb_ensemble = y_valid_proba_xgb

print(f"\nEnsemble 基礎模型:")
print(f"  1. {lgb_ensemble_name} (F1={lgb_ensemble_f1:.4f})")
print(f"  2. {xgb_ensemble_name} (F1={xgb_ensemble_f1:.4f})")

# 5.7.2 方法 1: Simple Average Ensemble
print("\n" + "-"*80)
print("Step 5.7.2: 方法 1 - Simple Average Ensemble")
print("-"*80)

print("\n計算預測機率的簡單平均...")
y_valid_proba_avg = (y_valid_proba_lgb_ensemble + y_valid_proba_xgb_ensemble) / 2
y_valid_pred_avg = (y_valid_proba_avg >= 0.5).astype(int)

avg_precision = precision_score(y_valid, y_valid_pred_avg, pos_label=1)
avg_recall = recall_score(y_valid, y_valid_pred_avg, pos_label=1)
avg_f1 = f1_score(y_valid, y_valid_pred_avg, pos_label=1)
avg_roc_auc = roc_auc_score(y_valid, y_valid_proba_avg)

print(f"\n驗證集性能 (Simple Average Ensemble):")
print(f"  Precision: {avg_precision:.4f}")
print(f"  Recall: {avg_recall:.4f}")
print(f"  F1-Score: {avg_f1:.4f}")
print(f"  ROC-AUC: {avg_roc_auc:.4f}")

# 5.7.3 方法 2: Weighted Average Ensemble (基於驗證集 F1)
print("\n" + "-"*80)
print("Step 5.7.3: 方法 2 - Weighted Average Ensemble (基於 F1-Score)")
print("-"*80)

# 計算權重（基於驗證集 F1-Score）
total_f1 = lgb_ensemble_f1 + xgb_ensemble_f1
lgb_weight = lgb_ensemble_f1 / total_f1
xgb_weight = xgb_ensemble_f1 / total_f1

print(f"\n權重分配:")
print(f"  LightGBM 權重: {lgb_weight:.4f} (基於 F1={lgb_ensemble_f1:.4f})")
print(f"  XGBoost 權重: {xgb_weight:.4f} (基於 F1={xgb_ensemble_f1:.4f})")

y_valid_proba_weighted = (lgb_weight * y_valid_proba_lgb_ensemble + 
                          xgb_weight * y_valid_proba_xgb_ensemble)
y_valid_pred_weighted = (y_valid_proba_weighted >= 0.5).astype(int)

weighted_precision = precision_score(y_valid, y_valid_pred_weighted, pos_label=1)
weighted_recall = recall_score(y_valid, y_valid_pred_weighted, pos_label=1)
weighted_f1 = f1_score(y_valid, y_valid_pred_weighted, pos_label=1)
weighted_roc_auc = roc_auc_score(y_valid, y_valid_proba_weighted)

print(f"\n驗證集性能 (Weighted Average Ensemble):")
print(f"  Precision: {weighted_precision:.4f}")
print(f"  Recall: {weighted_recall:.4f}")
print(f"  F1-Score: {weighted_f1:.4f}")
print(f"  ROC-AUC: {weighted_roc_auc:.4f}")

# 5.7.4 方法 3: Stacking Ensemble
print("\n" + "-"*80)
print("Step 5.7.4: 方法 3 - Stacking Ensemble (Meta-learner)")
print("-"*80)

from sklearn.linear_model import LogisticRegression as LR

print("\n訓練 Stacking 模型...")
print("  - 基礎模型: LightGBM + XGBoost")
print("  - Meta-learner: Logistic Regression")

# 創建 stacking 特徵（使用驗證集的預測機率）
X_valid_stack = np.column_stack([y_valid_proba_lgb_ensemble, y_valid_proba_xgb_ensemble])

# 使用訓練集生成預測來訓練 meta-learner
# ⚠️ 關鍵：兩個模型都在「原始訓練集 X_train」上生成預測
print("\n  生成訓練集的預測機率...")

# LightGBM 在原始訓練集上的預測
y_train_proba_lgb_meta = ensemble_lgb.predict_proba(X_train)[:, 1]
print(f"    LightGBM 預測: {len(y_train_proba_lgb_meta)} 筆")

# XGBoost 在原始訓練集上的預測
y_train_proba_xgb_meta = ensemble_xgb.predict_proba(X_train)[:, 1]
print(f"    XGBoost 預測: {len(y_train_proba_xgb_meta)} 筆")

# 訓練集的 stacking 特徵
X_train_stack = np.column_stack([y_train_proba_lgb_meta, y_train_proba_xgb_meta])
print(f"  ✓ Stacking 特徵矩陣: {X_train_stack.shape}")

# 訓練 meta-learner（使用原始訓練集的標籤）
meta_learner = LR(max_iter=1000, random_state=RANDOM_STATE)
meta_learner.fit(X_train_stack, y_train)  # ⭐ 使用原始 y_train
print("✓ Meta-learner 訓練完成")

# 在驗證集上預測
y_valid_proba_stack = meta_learner.predict_proba(X_valid_stack)[:, 1]
y_valid_pred_stack = meta_learner.predict(X_valid_stack)

stack_precision = precision_score(y_valid, y_valid_pred_stack, pos_label=1)
stack_recall = recall_score(y_valid, y_valid_pred_stack, pos_label=1)
stack_f1 = f1_score(y_valid, y_valid_pred_stack, pos_label=1)
stack_roc_auc = roc_auc_score(y_valid, y_valid_proba_stack)

print(f"\n驗證集性能 (Stacking Ensemble):")
print(f"  Precision: {stack_precision:.4f}")
print(f"  Recall: {stack_recall:.4f}")
print(f"  F1-Score: {stack_f1:.4f}")
print(f"  ROC-AUC: {stack_roc_auc:.4f}")

# 5.7.5 Ensemble 方法比較
print("\n" + "="*80)
print("Ensemble 方法比較:")
print("="*80)

ensemble_comparison = pd.DataFrame({
    'Method': [
        lgb_ensemble_name,
        xgb_ensemble_name,
        'Simple Average',
        'Weighted Average',
        'Stacking'
    ],
    'Type': [
        'Single Model',
        'Single Model',
        'Ensemble',
        'Ensemble',
        'Ensemble'
    ],
    'F1-Score': [
        lgb_ensemble_f1,
        xgb_ensemble_f1,
        avg_f1,
        weighted_f1,
        stack_f1
    ],
    'Precision': [
        precision_score(y_valid, ensemble_lgb.predict(X_valid if 'class_weight' in lgb_ensemble_name else X_valid), pos_label=1),
        precision_score(y_valid, ensemble_xgb.predict(X_valid if 'scale_pos_weight' in xgb_ensemble_name else X_valid), pos_label=1),
        avg_precision,
        weighted_precision,
        stack_precision
    ],
    'Recall': [
        recall_score(y_valid, ensemble_lgb.predict(X_valid if 'class_weight' in lgb_ensemble_name else X_valid), pos_label=1),
        recall_score(y_valid, ensemble_xgb.predict(X_valid if 'scale_pos_weight' in xgb_ensemble_name else X_valid), pos_label=1),
        avg_recall,
        weighted_recall,
        stack_recall
    ],
    'ROC-AUC': [
        roc_auc_score(y_valid, y_valid_proba_lgb_ensemble),
        roc_auc_score(y_valid, y_valid_proba_xgb_ensemble),
        avg_roc_auc,
        weighted_roc_auc,
        stack_roc_auc
    ]
})

print("\n" + ensemble_comparison.to_string(index=False))

# 選擇最佳 Ensemble 方法
ensemble_f1_scores = [avg_f1, weighted_f1, stack_f1]
ensemble_methods = ['Simple Average', 'Weighted Average', 'Stacking']
best_ensemble_idx = np.argmax(ensemble_f1_scores)
best_ensemble_method = ensemble_methods[best_ensemble_idx]
best_ensemble_f1 = ensemble_f1_scores[best_ensemble_idx]

print(f"\n🏆 最佳 Ensemble 方法: {best_ensemble_method}")
print(f"   F1-Score: {best_ensemble_f1:.4f}")

# 與單一模型比較
print(f"\nEnsemble vs 最佳單一模型:")
print(f"  最佳單一模型: {best_model_name} (F1={all_f1_scores[best_model_idx]:.4f})")
print(f"  最佳 Ensemble: {best_ensemble_method} (F1={best_ensemble_f1:.4f})")

if best_ensemble_f1 > all_f1_scores[best_model_idx]:
    improvement = (best_ensemble_f1 - all_f1_scores[best_model_idx]) * 100
    print(f"  → Ensemble 較好！提升 {improvement:.2f}% ✅")
    
    # 更新最佳模型為 Ensemble
    best_model_name = f"Ensemble ({best_ensemble_method})"
    
    # 根據選擇的 ensemble 方法設定預測函數
    if best_ensemble_method == 'Simple Average':
        final_ensemble_proba = y_valid_proba_avg
    elif best_ensemble_method == 'Weighted Average':
        final_ensemble_proba = y_valid_proba_weighted
    else:  # Stacking
        final_ensemble_proba = y_valid_proba_stack
else:
    print(f"  → 單一模型較好")

# 視覺化 Ensemble 比較
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# F1-Score 比較
methods = ensemble_comparison['Method'].tolist()
f1_scores = ensemble_comparison['F1-Score'].tolist()
colors_ensemble = ['#e74c3c', '#2ecc71', '#3498db', '#f39c12', '#9b59b6']

axes[0].bar(range(len(methods)), f1_scores, color=colors_ensemble)
axes[0].set_xticks(range(len(methods)))
axes[0].set_xticklabels(methods, rotation=15, ha='right')
axes[0].set_title('F1-Score Comparison: Single Models vs Ensemble', fontsize=14, fontweight='bold')
axes[0].set_ylabel('F1-Score', fontsize=12)
axes[0].set_ylim([min(f1_scores) - 0.01, max(f1_scores) + 0.01])
axes[0].grid(alpha=0.3, axis='y')
axes[0].axhline(y=max(f1_scores), color='red', linestyle='--', alpha=0.5, label='Best Performance')
for i, v in enumerate(f1_scores):
    axes[0].text(i, v + 0.001, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)
axes[0].legend()

# ROC-AUC 比較
roc_scores = ensemble_comparison['ROC-AUC'].tolist()

axes[1].bar(range(len(methods)), roc_scores, color=colors_ensemble)
axes[1].set_xticks(range(len(methods)))
axes[1].set_xticklabels(methods, rotation=15, ha='right')
axes[1].set_title('ROC-AUC Comparison: Single Models vs Ensemble', fontsize=14, fontweight='bold')
axes[1].set_ylabel('ROC-AUC', fontsize=12)
axes[1].set_ylim([min(roc_scores) - 0.005, max(roc_scores) + 0.005])
axes[1].grid(alpha=0.3, axis='y')
axes[1].axhline(y=max(roc_scores), color='red', linestyle='--', alpha=0.5, label='Best Performance')
for i, v in enumerate(roc_scores):
    axes[1].text(i, v + 0.0005, f'{v:.4f}', ha='center', fontweight='bold', fontsize=9)
axes[1].legend()

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '03_ensemble_comparison.png'), dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ Ensemble 比較圖已保存: {OUTPUT_DIR}/03_ensemble_comparison.png")

# =========================
# Step 6: 在驗證集上評估最佳模型（可能是 Ensemble）
# =========================

print("\n" + "="*80)
print("Step 6: 在驗證集上評估最佳模型")
print("="*80)

print(f"\n最佳模型: {best_model_name}")

# 根據最佳模型類型進行預測
if 'Ensemble' in best_model_name:
    # 使用 Ensemble 預測
    y_valid_proba = final_ensemble_proba
    y_valid_pred = (y_valid_proba >= 0.5).astype(int)
else:
    # 使用單一模型預測
    y_valid_pred = final_model.predict(X_valid)
    y_valid_proba = final_model.predict_proba(X_valid)[:, 1]

# 評估指標
cm_valid = confusion_matrix(y_valid, y_valid_pred)
print("\n驗證集混淆矩陣:")
print(cm_valid)

print("\nClassification Report (驗證集):")
print(classification_report(y_valid, y_valid_pred, target_names=['Normal', 'Fraud']))

valid_precision = precision_score(y_valid, y_valid_pred, pos_label=1)
valid_recall = recall_score(y_valid, y_valid_pred, pos_label=1)
valid_f1 = f1_score(y_valid, y_valid_pred, pos_label=1)
valid_roc_auc = roc_auc_score(y_valid, y_valid_proba)

print(f"\n驗證集詳細指標 (Class=1):")
print(f"  Precision: {valid_precision:.4f}")
print(f"  Recall: {valid_recall:.4f}")
print(f"  F1-Score: {valid_f1:.4f}")
print(f"  ROC-AUC: {valid_roc_auc:.4f}")

# =========================
# Step 7: 閾值調整 (在驗證集上決定) ⭐
# =========================

print("\n" + "="*80)
print("Step 7: 閾值調整 (嚴謹版本 - 在驗證集上決定最佳閾值)")
print("="*80)

print("\n⚠️ 重要: 在驗證集上找最佳閾值，測試集只用來最終評估")

# 7-1. 在驗證集上掃描閾值
print("\n在驗證集上掃描閾值...")
thresholds = np.arange(0.1, 0.9, 0.05)
f1_scores_threshold = []
precision_scores = []
recall_scores = []

for threshold in thresholds:
    y_pred_threshold = (y_valid_proba >= threshold).astype(int)
    f1 = f1_score(y_valid, y_pred_threshold, pos_label=1)
    precision = precision_score(y_valid, y_pred_threshold, pos_label=1)
    recall = recall_score(y_valid, y_pred_threshold, pos_label=1)
    
    f1_scores_threshold.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)

# 7-2. 找出最佳閾值
best_threshold_idx = np.argmax(f1_scores_threshold)
best_threshold = thresholds[best_threshold_idx]
best_valid_f1 = f1_scores_threshold[best_threshold_idx]
best_valid_precision = precision_scores[best_threshold_idx]
best_valid_recall = recall_scores[best_threshold_idx]

print(f"\n✓ 在驗證集上找到最佳閾值")
print(f"\n最佳閾值: {best_threshold:.2f}")
print(f"  驗證集 Precision: {best_valid_precision:.4f}")
print(f"  驗證集 Recall: {best_valid_recall:.4f}")
print(f"  驗證集 F1-Score: {best_valid_f1:.4f}")

# 7-3. 視覺化閾值調整過程
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(thresholds, precision_scores, 'b-', label='Precision', linewidth=2)
ax.plot(thresholds, recall_scores, 'g-', label='Recall', linewidth=2)
ax.plot(thresholds, f1_scores_threshold, 'r-', label='F1-Score', linewidth=2)
ax.axvline(x=best_threshold, color='orange', linestyle='--', linewidth=2, 
           label=f'Best Threshold = {best_threshold:.2f}')
ax.set_xlabel('Threshold', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Threshold Tuning on Validation Set', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.savefig(os.path.join(OUTPUT_DIR, '05_threshold_tuning_validation.png'), dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ 閾值調整圖表已保存: {OUTPUT_DIR}/05_threshold_tuning_validation.png")

# =========================
# Step 8: 最終在測試集上評估（使用固定閾值）⭐
# =========================

print("\n" + "="*80)
print("Step 8: 最終測試集評估 (使用固定閾值)")
print("="*80)

print(f"\n使用在驗證集上決定的最佳閾值: {best_threshold:.2f}")
print("⚠️ 測試集只評估一次，不再調整任何參數")

# 8-1. 在測試集上預測
if 'Ensemble' in best_model_name:
    # 使用 Ensemble 預測
    # 先獲取基礎模型的預測
    y_test_proba_lgb_base = ensemble_lgb.predict_proba(X_test)[:, 1]
    y_test_proba_xgb_base = ensemble_xgb.predict_proba(X_test)[:, 1]
    
    if best_ensemble_method == 'Simple Average':
        y_test_proba = (y_test_proba_lgb_base + y_test_proba_xgb_base) / 2
    elif best_ensemble_method == 'Weighted Average':
        y_test_proba = (lgb_weight * y_test_proba_lgb_base + 
                       xgb_weight * y_test_proba_xgb_base)
    else:  # Stacking
        X_test_stack = np.column_stack([y_test_proba_lgb_base, y_test_proba_xgb_base])
        y_test_proba = meta_learner.predict_proba(X_test_stack)[:, 1]
else:
    # 使用單一模型預測
    y_test_proba = final_model.predict_proba(X_test)[:, 1]

y_test_pred = (y_test_proba >= best_threshold).astype(int)  # ⭐ 使用固定閾值

# 8-2. 評估指標
test_precision = precision_score(y_test, y_test_pred, pos_label=1)
test_recall = recall_score(y_test, y_test_pred, pos_label=1)
test_f1 = f1_score(y_test, y_test_pred, pos_label=1)
test_roc_auc = roc_auc_score(y_test, y_test_proba)

print("\n" + "-"*80)
print("測試集最終結果:")
print("-"*80)

# 混淆矩陣
cm_test = confusion_matrix(y_test, y_test_pred)
print("\n測試集混淆矩陣:")
print(cm_test)

# 視覺化混淆矩陣
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=['Normal', 'Fraud'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title(f'{best_model_name} - Confusion Matrix (Test Set)', fontsize=14, fontweight='bold')
plt.savefig(os.path.join(OUTPUT_DIR, '06_confusion_matrix_test.png'), dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ 混淆矩陣已保存: {OUTPUT_DIR}/06_confusion_matrix_test.png")

# Classification Report
print("\nClassification Report (測試集):")
print(classification_report(y_test, y_test_pred, target_names=['Normal', 'Fraud']))

print(f"\n測試集詳細指標 (Class=1):")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall: {test_recall:.4f}")
print(f"  F1-Score: {test_f1:.4f}")
print(f"  ROC-AUC: {test_roc_auc:.4f}")

# =========================
# Step 9: 模型解釋 (SHAP & LIME)
# =========================

print("\n" + "="*80)
print("Step 9: 模型解釋 (SHAP & LIME)")
print("="*80)

# 9-1. SHAP 解釋
print("\n使用 SHAP 進行模型解釋...")

# 使用測試集的一部分（200 筆）
X_test_sample = X_test.iloc[:200]
print(f"  使用 {len(X_test_sample)} 筆測試資料進行 SHAP 分析")

# 選擇用於 SHAP 解釋的模型
if 'Ensemble' in best_model_name:
    print(f"  ⚠️ 最佳模型是 Ensemble，使用 {lgb_ensemble_name} 進行 SHAP 分析")
    shap_model = ensemble_lgb
    shap_model_name = lgb_ensemble_name
else:
    shap_model = final_model
    shap_model_name = best_model_name

# 建立 SHAP explainer
if 'LightGBM' in shap_model_name or 'XGBoost' in shap_model_name:
    explainer = shap.TreeExplainer(shap_model)
    shap_values = explainer.shap_values(X_test_sample)
    
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

print("  ✓ SHAP 值計算完成")

# SHAP Summary Plot (條狀圖)
print("\n  繪製 SHAP 特徵重要性圖...")
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, plot_type="bar", show=False, max_display=20)
plt.title('SHAP Feature Importance (Top 20)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '07_shap_importance.png'), dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ SHAP 特徵重要性圖已保存: {OUTPUT_DIR}/07_shap_importance.png")

# SHAP Summary Plot (點圖)
print("\n  繪製 SHAP 摘要圖...")
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test_sample, show=False, max_display=20)
plt.title('SHAP Feature Impact Summary (Top 20)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, '08_shap_summary.png'), dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ SHAP 摘要圖已保存: {OUTPUT_DIR}/08_shap_summary.png")

# 9-2. LIME 解釋
print("\n使用 LIME 進行個別交易解釋...")

# 建立 LIME explainer
# 根據模型類型選擇訓練資料
if 'Ensemble' in best_model_name:
    if 'class_weight' in lgb_ensemble_name or 'scale_pos_weight' in lgb_ensemble_name:
        lime_train_data = X_train.values
    else:
        lime_train_data = X_train_smote.values
    lime_feature_names = X_train.columns.tolist()
else:
    if best_model_name in ['LightGBM (class_weight)', 'XGBoost (scale_pos_weight)']:
        lime_train_data = X_train.values
        lime_feature_names = X_train.columns.tolist()
    else:
        lime_train_data = X_train_smote.values
        lime_feature_names = X_train_smote.columns.tolist()

lime_explainer = lime_tabular.LimeTabularExplainer(
    lime_train_data,
    feature_names=lime_feature_names,
    class_names=['Normal', 'Fraud'],
    mode='classification',
    random_state=RANDOM_STATE
)

# 選擇詐欺和正常交易各一筆
fraud_idx = y_test[y_test == 1].index[0]
normal_idx = y_test[y_test == 0].index[0]

# 定義預測函數（支持 Ensemble）
def lime_predict_fn(X_data):
    if 'Ensemble' in best_model_name:
        # Ensemble 預測
        proba_lgb = ensemble_lgb.predict_proba(X_data)
        proba_xgb = ensemble_xgb.predict_proba(X_data)
        
        if best_ensemble_method == 'Simple Average':
            return (proba_lgb + proba_xgb) / 2
        elif best_ensemble_method == 'Weighted Average':
            return lgb_weight * proba_lgb + xgb_weight * proba_xgb
        else:  # Stacking
            X_stack = np.column_stack([proba_lgb[:, 1], proba_xgb[:, 1]])
            return meta_learner.predict_proba(X_stack)
    else:
        # 單一模型預測
        return final_model.predict_proba(X_data)

print(f"\n  解釋詐欺交易 (索引: {fraud_idx})...")
exp_fraud = lime_explainer.explain_instance(
    X_test.loc[fraud_idx].values,
    lime_predict_fn,
    num_features=10
)

print(f"  解釋正常交易 (索引: {normal_idx})...")
exp_normal = lime_explainer.explain_instance(
    X_test.loc[normal_idx].values,
    lime_predict_fn,
    num_features=10
)

# 儲存 LIME 解釋
exp_fraud.save_to_file(os.path.join(OUTPUT_DIR, '09_lime_fraud.html'))
exp_normal.save_to_file(os.path.join(OUTPUT_DIR, '10_lime_normal.html'))

print(f"  ✓ LIME 解釋已保存:")
print(f"    - 詐欺交易: {OUTPUT_DIR}/09_lime_fraud.html")
print(f"    - 正常交易: {OUTPUT_DIR}/10_lime_normal.html")

# 9-3. 特徵重要性分析
print("\n" + "-"*80)
print("模型解釋分析:")
print("-"*80)

shap_importance = np.abs(shap_values).mean(axis=0)
top_features_idx = np.argsort(shap_importance)[::-1][:10]
top_features = X_test_sample.columns[top_features_idx]

print("\n最重要的 10 個特徵 (依照 SHAP 重要性):")
for i, feat in enumerate(top_features, 1):
    print(f"  {i}. {feat}")

# =========================
# Step 10: 整體總結與輸出
# =========================

print("\n" + "="*80)
print("Step 10: 整體總結與輸出")
print("="*80)

# 10-1. 總結報告
print("\n" + "="*80)
print("🎯 最終模型總結報告 (嚴謹版本)")
print("="*80)

print(f"\n最佳模型: {best_model_name}")
print(f"不平衡處理方式: SMOTE")
print(f"閾值: {best_threshold:.2f} (在驗證集上決定)")

print(f"\n驗證集性能 (閾值={best_threshold:.2f}):")
print(f"  - Precision: {best_valid_precision:.4f}")
print(f"  - Recall: {best_valid_recall:.4f}")
print(f"  - F1-Score: {best_valid_f1:.4f}")

print(f"\n測試集性能 (使用固定閾值={best_threshold:.2f}):")
print(f"  - Precision: {test_precision:.4f}")
print(f"  - Recall: {test_recall:.4f}")
print(f"  - F1-Score: {test_f1:.4f}")
print(f"  - ROC-AUC: {test_roc_auc:.4f}")

print("\n嚴謹性改進:")
print("  ✅ V1~V28 是 PCA 特徵，已標準化，不需額外 scaling")
print("  ✅ 加入 Logistic Regression baseline")
print("  ✅ 閾值在驗證集上決定，測試集只評估一次")

print("\n關鍵發現:")
print(f"  1. 模型成功捕捉到 {test_recall*100:.2f}% 的詐欺交易")
print(f"  2. 預測為詐欺的交易中，有 {test_precision*100:.2f}% 是真的詐欺")
print(f"  3. 測試集 F1-Score: {test_f1:.4f}")

# 10-2. 保存預測結果
print("\n" + "-"*80)
print("保存預測結果...")
print("-"*80)

# 建立結果 DataFrame
results_df = X_test.copy()
results_df['Actual_Class'] = y_test
results_df['Predicted_Class'] = y_test_pred
results_df['Fraud_Probability'] = y_test_proba
results_df['Correct_Prediction'] = (results_df['Actual_Class'] == results_df['Predicted_Class'])

# 保存到 CSV
results_df.to_csv(os.path.join(OUTPUT_DIR, 'prediction_results.csv'), index=False)
print(f"✓ 預測結果已保存: {OUTPUT_DIR}/prediction_results.csv")

print(f"\n檔案包含 {len(results_df)} 筆測試資料的預測結果")

# 統計預測正確率
accuracy = results_df['Correct_Prediction'].mean()
print(f"整體準確率: {accuracy*100:.2f}%")

# 10-3. 生成檔案清單
print("\n" + "-"*80)
print("生成的檔案清單:")
print("-"*80)

files = [
    '01_time_distribution.png',
    '02_amount_distribution.png',
    '03_model_comparison_all.png',
    '03_ensemble_comparison.png',
    '04_confusion_matrices_comparison.png',
    '05_threshold_tuning_validation.png',
    '06_confusion_matrix_test.png',
    '07_shap_importance.png',
    '08_shap_summary.png',
    '09_lime_fraud.html',
    '10_lime_normal.html',
    'prediction_results.csv'
]

for i, file in enumerate(files, 1):
    print(f"  {i}. {OUTPUT_DIR}/{file}")

print("\n" + "="*80)
print("✅ 所有步驟完成！(嚴謹版本)")
print("="*80)

print(f"\n執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n感謝使用！這個版本更符合機器學習最佳實踐。")
print("="*80)


Step 0: 基本設定與資料讀取

⚙️ Optuna 調參次數設定: 20 trials
   💡 提示：
      - 10 trials: 快速測試（約 5-10 分鐘）
      - 20 trials: 平衡選擇（約 10-20 分鐘）⭐ 推薦
      - 50 trials: 深度搜索（約 25-40 分鐘）
      - 100 trials: 極致優化（約 50-80 分鐘）

✓ 所有套件匯入成功
✓ 隨機種子設定為: 42
✓ 輸出路徑: outputs

正在讀取資料: data/raw/creditcard.csv


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/creditcard.csv'